### train car 3

# 전처리된 훈련, 테스트 데이터를 읽고 학습한다.

In [1]:
import sys
import pandas as pd

from keras.applications import mobilenetv2
from keras.preprocessing import image
# import cv2
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense, Softmax
import random
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import TensorBoard
from keras.models import Input, Model

from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns

Using TensorFlow backend.


In [2]:
modelpath='car.h5'
cache=False
basedir = './'

x_trainall = np.load('x_train.npy')
y_trainall = np.load('y_train.npy')
dfclass = pd.read_csv(basedir+'class.csv')

In [3]:
get_ipython().run_line_magic('matplotlib', 'inline')

In [4]:
imgwidth=224
imgheight=224

In [5]:
x_train = x_trainall[:8000]
y_train = y_trainall[:8000]
x_val = x_trainall[8000:]
y_val = y_trainall[8000:]
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(8000, 224, 224, 3) (8000,) (1990, 224, 224, 3) (1990,)


In [6]:
y_train_onehot = np_utils.to_categorical(y_train, 197)
y_val_onehot = np_utils.to_categorical(y_val, 197)

In [7]:
# Image Augumentation
batch_size=32
datagen1 = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')
datagen2 = ImageDataGenerator( rescale=1./255)
train_generator = datagen1.flow(x_train, y_train_onehot, batch_size=batch_size)
val_generator = datagen2.flow(x_val, y_val_onehot, batch_size=batch_size)

In [8]:
tensorboard = TensorBoard(log_dir='log/'+str(time.time()))
if os.path.exists(modelpath):
    cache=True
    print('cache model use... continuous!')
else:
    cache=False
    print('no cache model. create new model!')

cache model use... continuous!


In [9]:
# model
model = load_model(modelpath)
print('load model...')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
load model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
_______________________________________________________________________________________________

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#epochs = 200
# hist = model.fit( x_train, y_train_onehot, batch_size=50, shuffle=True,  epochs=epochs, 
#                  verbose=1 , callbacks=[tensorboard], validation_data=[x_val, y_val_onehot] )
# 5, 
hist = model.fit_generator( train_generator, epochs = 200, validation_data=val_generator, 
                           steps_per_epoch=len(x_train)/32, validation_steps=len(x_val)/32)

Epoch 1/200
250/250 [==============================] - 51s 203ms/step - loss: 0.3647 - acc: 0.8896 - val_loss: 1.3657 - val_acc: 0.6769
Epoch 2/200
250/250 [==============================] - 49s 197ms/step - loss: 0.4326 - acc: 0.8659 - val_loss: 1.7275 - val_acc: 0.6080
Epoch 3/200
250/250 [==============================] - 49s 195ms/step - loss: 0.4351 - acc: 0.8688 - val_loss: 3.4766 - val_acc: 0.3995
Epoch 4/200
250/250 [==============================] - 50s 200ms/step - loss: 0.3957 - acc: 0.8760 - val_loss: 2.5602 - val_acc: 0.5332
Epoch 5/200
250/250 [==============================] - 49s 198ms/step - loss: 0.3004 - acc: 0.9062 - val_loss: 4.0725 - val_acc: 0.3889
Epoch 6/200
241/250 [===========================>..] - ETA: 1s - loss: 0.3517 - acc: 0.8912

In [ ]:

# hist = model.evaluate( x_val/255., y_val_onehot, batch_size=30, verbose=1 )
# or 
hist = model.evaluate_generator(val_generator, steps=len(x_val)/32)
print(hist)

In [ ]:
modelpath='car.h5'
model.save(modelpath)

In [ ]:
# submission
x_test = np.load('x_test.npy')
predictions = model.predict( x_test/255. )
pdi = np.argmax(predictions, axis=1)
print(pdi, np.min(pdi), np.max(pdi))


In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission["class"] = pdi
submission.to_csv("submission.csv", index=False)
submission.head()


In [ ]:
sns.countplot(submission["class"], order=submission["class"].value_counts(ascending=True).index)